# Entregable1- IRN
### Grupo 3
### Integrantes:
Diego Martín Vásquez Martínez


# Código de Generación de Dataset


### Primera Parte:
#### En esta parte se instalan e importan las libreías necesarias para el correcto funcionamiento del código, así como también se definen las rutas de archivos de interés.

In [ ]:
pip install -r requirements.txt 

In [ ]:
pip install easyocr

In [ ]:
import cv2
import os
import easyocr
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import subprocess
from dotenv import load_dotenv, find_dotenv
import PyPDF2
import os
from PIL import Image
from difflib import get_close_matches
rutaTexto=r"DataTexto"
load_dotenv(find_dotenv(),override = True)


Transformacion de archivos de PDF Y PNG a TXT

### Segunda parte:
#### Utilización de easyOCR y PyPDF2 para la transformación de archivos .PNG y .PDF a archivos .txt

In [ ]:
def IMGtoText(ruta_img, caso_id):
    reader= easyocr.Reader(["es"], gpu= False)
    image=cv2.imread(ruta_img)
    result= reader.readtext(image, paragraph= True)
    texto_detectado = [resultado[1] for resultado in result]
    texto_completo = ' '.join(texto_detectado)

   #Definir la ruta y el nombre del archivo de texto resultante
    carpeta, nombre_img = os.path.split(ruta_img)
    nombre_txt = caso_id + "_" + os.path.splitext(nombre_img)[0] + "_SEGUIMIENTO_EXPEDIENTE.txt"
    ruta_txt = os.path.join(carpeta, nombre_txt)
    with open(ruta_txt, 'w') as archivo:
         archivo.write(texto_completo)




    
 

In [ ]:
#Convierte los archivos PDFs a texto.
def PDFtoText(ruta_pdf,caso_id):
    pdf= open(ruta_pdf,"rb")
    reader= PyPDF2.PdfReader(pdf)
    extract_info= ""
    for num in range(len(reader.pages)):
        info_page = reader._get_page(num)
        extract_info= extract_info + info_page.extract_text()

    #Crea la ruta del archivo donde se almacena el texto plano.
    carpeta, nombre_pdf = os.path.split(ruta_pdf)
    nombre_txt = caso_id +"_" + os.path.splitext(nombre_pdf)[0] + ".txt"

    ruta_txt = os.path.join(carpeta, nombre_txt)

    if extract_info != "":
        with open(ruta_txt, 'w') as archivo:
            archivo.write(extract_info)

In [ ]:
def DeterminarID(string):
    indice = 0
    # Elimina los valores iniciales no numéricos del nombre, como 'exp', etc.
    while indice < len(string) and not string[indice].isdigit():
        indice += 1
    string = string[indice:]
    #Dado que el nombre del archivo incluye el estado, se puede obtener el estado de manera deterministica.
    #Este estado deterministico será utilizado solamente para hallar la metrica accuracy
    id_code, estado = string.rsplit('-', 1)
    return id_code,estado

In [ ]:
def ProcesarCarpeta(rutaPrimaria, nombre_carpeta):
    ruta_carpeta_completa = os.path.join(rutaPrimaria, nombre_carpeta)
    lista_archivos = os.listdir(ruta_carpeta_completa)

    for nombre_archivo in lista_archivos:
        ruta_archivo = os.path.join(ruta_carpeta_completa, nombre_archivo)
        caso_id, estado= DeterminarID(nombre_carpeta)
        if ruta_archivo.lower().endswith('.pdf'):
            try:
                PDFtoText(ruta_archivo,caso_id)
                print('Se convirtio PDF')
            except Exception as e:
                print(f"Error al convertir el PDF: {e}")

        elif ruta_archivo.lower().endswith('.png'):
            IMGtoText(ruta_archivo,caso_id)
            print('Se convirtio Imagen')

        else:
            print(f"{nombre_archivo} no es ni PDF ni imagen")

In [ ]:
#En esta celda, se corren las funciones de manera adecuada para lograr obtener los archivos en texto plano
rutaTexto= rutaTexto
lista_carpeta = [nombre for nombre in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, nombre))]
for carpeta in lista_carpeta:
    ruta = os.path.join(rutaTexto,carpeta)
    lista_carpetas = os.listdir(ruta)
    for nombre in lista_carpetas:
        ProcesarCarpeta(ruta, nombre)
        ruta_expediente= os.path.join(ruta,nombre)

    # Imprimir
    print("Lista de nombres de carpetas:")
    print(lista_carpetas)

### Tercera Parte:
#### Se utilizan modelos de LLM, a los cuales se les darán los archivos y ciertos prompts, para obtener el overview y el estado del caso.

In [ ]:
def ia(texto, humanInput):
    llm  = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
    docs = [Document(page_content=texto)]

    template = humanInput + '''
    del siguiente texto:
    TEXTO:{text}.
    '''
    prompt = PromptTemplate(
        input_variables=['text'],
        template=template
    )
    chain = load_summarize_chain(
        llm,
        chain_type='stuff',
        prompt=prompt,
        verbose=False
    )
    resumen = chain.run(docs)
    return resumen

Ahora, se probaran distintos promts, ademas de distintas formas de introducir la data.

El primer método de introducir la información es combinando toda la info en un solo string y dándosela de frente a la IA


In [ ]:
def IA1(rutaExpediente,resumenPrompt,estadoPrompt):
    texto_ia= ""
    for archivo in os.listdir(rutaExpediente):
        if archivo.lower().endswith(".txt"):  # Asegúrate de que solo estés leyendo archivos .txt
            ruta_archivo = os.path.join(rutaExpediente, archivo)
            with open(ruta_archivo, 'r') as file:
                contenido = file.read()
                texto_ia += contenido  # Agregar el contenido de cada archivo a la variable de texto combinado

# Imprimir o hacer cualquier operación que necesites con el texto combinado
    overview = ia(texto_ia,resumenPrompt)
    estado= ia(texto_ia,estadoPrompt)
    print("OVERVIEW: "+overview)
    print("Estado: "+ estado)
    return overview, estado

El segundo método es darle a la IA archivo por archivo, que saque un resumen por archivo y lo una formando una string, y finalmente sacar un resumen de dicha string unida

In [ ]:

def IA2(rutaExpediente,resumenPrompt,estadoPrompt):

    overview= ""
    estado= ""
    for archivo in os.listdir(rutaExpediente):
        if archivo.lower().endswith(".txt"):  # Asegúrate de que solo estés leyendo archivos .txt
            ruta_archivo = os.path.join(rutaExpediente, archivo)
            with open(ruta_archivo, 'r') as file:
                texto_ia = file.read()
            overview= overview + ia(texto_ia,resumenPrompt)
            estado= estado+ ia(texto_ia,estadoPrompt)


    overview= ia(overview,resumenPrompt)
    estado= ia(estado,estadoPrompt)
    print("OVERVIEW: "+overview)
    print("Estado: "+ estado)

El tercer método es un método regresivo, lo que hace es que cada resumen lo introduce nuevamente junto al siguiente archivo no resumido.

In [ ]:
def IA3(rutaExpediente,resumenPrompt,estadoPrompt):

    overview= ""
    estado= ""
    for archivo in os.listdir(rutaExpediente):
        if archivo.lower().endswith(".txt"):  # Asegúrate de que solo estés leyendo archivos .txt
            ruta_archivo = os.path.join(rutaExpediente, archivo)
            with open(ruta_archivo, 'r') as file:
                overview = overview + file.read()
                estado= estado + file.read()
            overview= ia(overview,resumenPrompt)
            estado= ia(estado,estadoPrompt)

    print("OVERVIEW: "+overview)
    print("Estado: "+ estado)

In [ ]:
def Escritura(ruta,caso_id, overview, estado,estado_deterministico):
    #Generamos la Ruta de la carpeta.
    if not os.path.exists(ruta):
        os.makedirs(ruta)
    carpeta = "Archivos_IA_"+caso_id
    ruta_carpeta = os.path.join(ruta, carpeta)
    if not os.path.exists(ruta_carpeta):
        os.makedirs(ruta_carpeta)

    #Generamos la ruta de los archivos.
    ruta_overview = os.path.join(ruta_carpeta,caso_id+ "_overview.txt")
    ruta_estado = os.path.join(ruta_carpeta, caso_id+ "_estado.txt")
    ruta_estado_deterministico = os.path.join(ruta_carpeta, caso_id+ "_estado_deterministico.txt")


    #Crear los archivos
    with open(ruta_overview, 'w') as archivo:
        archivo.write(overview)
    with open(ruta_estado, 'w') as archivo:
        archivo.write(estado)
    with open(ruta_estado_deterministico, 'w') as archivo:
        archivo.write(estado_deterministico)
    print(f"___________________________________")

### Cuarta Parte:
#### Esta parte nos permite ejecutar las funciones declaradas de manera adecuada, lo que generará el overview y el estado del caso a través de modelos de LLM, de esta manera se logra obtener los archivos pedidos en el apartado B. del entregable 1.
#### En este caso, el método final escogido para introducirle data al modelo fue la funcion IA1, debido a que su tiempo de proceso fue muchísimo más corto que el resto de funciones.  

In [ ]:
rutaTexto = rutaTexto
resumenPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado son documentos legales de los cuales harás un resumen detallando las personas implicadas, y que es lo que demanda o defiende cada uno."
estadoPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado es una serie de documentos legales. Determina el estado del caso, sino puedes determinarlo responde en blanco o espacios, asegúrate que tu respuesta sea única y exclusivamente el estado del caso. Es decir, tu respuesta debe limitarse a: RECHAZADO, INADMISIBLE, EJECUCION, TRAMITE, PLAZO DE IMPUGNACION, CALIFICACION, ARCHIVO DEFINITIVO, ADMITIDA, ATENDIDO, SENTENCIADO, RESUELTO "
lista_carpetas= [carpeta for carpeta in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, carpeta))]
for carpeta in lista_carpetas:
    ruta_carpeta= os.path.join(rutaTexto, carpeta)
    lista_expedientes= [expediente for expediente in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, expediente))]
    for expediente in lista_expedientes:
        ruta_expediente= os.path.join(ruta_carpeta, expediente)
        caso_id, estado_deterministico = DeterminarID(expediente)
        overview,estado = IA1(ruta_expediente,resumenPrompt,estadoPrompt)
        Escritura(ruta_expediente,caso_id, overview, estado,estado_deterministico)
        print("__________________________________________")

# Expermimentación y documentación 
## Para la experimentación y documentación, se procesarán algunos casos (los cuales han sido escogidos aleatoriamente) y se compararán los resultados y el tiempo en el que se han obtenido. Se usarán las distintas funciones y diferentes prompts. 

## Experimentacion utilizando la funcion IA1():


### PROMPTS:
resumenPrompt= "Haz un resumen porfavor"
estadoPrompt= "Determina el estado del caso porfavor"

### Resultados: 
OVERVIEW: La demandante, Johana Paola Gomez Lescano, presenta una demanda de filiación judicial de paternidad extramatrimonial contra Luis Alberto Valladolid Camacho. Solicita que el demandado reconozca a su hijo Dayron Jesús Valladolid Gomez y que se le ordene pagar una pensión de alimentos mensual y adelantada de S/. 1,000.00 soles. También solicita una medida de asignación anticipada de alimentos. La demandante fundamenta su demanda en el hecho de que el demandado se ha negado a reconocer a su hijo y en la necesidad de su hijo de recibir alimentos y cuidados. Presenta como pruebas la partida de nacimiento del menor y el documento nacional de identidad del menor. La demandante solicita que se admita la demanda y se declare fundada en todos sus extremos.
Estado: El estado del caso es que la demanda ha sido admitida y se ha emitido una resolución para notificar al demandado.
Se han creado el archivos en texto plano.
OVERVIEW: El texto menciona que se trata de un seguimiento de expediente en el Poder Judicial del Perú, específicamente en la Corte Superior de Justicia de Piura, en la sede de Chipe. El expediente tiene el número 00201-2022-0-2001-JP-FC-07 y pertenece al Juzgado de Paz Letrado de Chipe. La fecha de inicio del proceso es el 3 de febrero de 2022 a las 22:38:38. El motivo de la demanda no se especifica en el resumen. El juez encargado del caso es Elías Lequernaque Carmen Irina Ocaña Malpartida Juan Manuel. El proceso se refiere a la materia de alimentos y se encuentra en estado de incidente. La ubicación actual del expediente no se menciona. Se interpone una demanda de alimentos adjuntando elementos de prueba. Los sujetos procesales son el demandado natural Noe Cruz Chinchay y la demandante natural Magaly del Milagro Cruz Flores.
Estado: No es posible determinar el estado del caso con la información proporcionada.
Se han creado el archivos en texto plano.
OVERVIEW: El texto es una demanda de alimentos presentada por María del Socorro Saucedo Saavedra ante el Juzgado de Paz Letrado de Piura. En la demanda, María del Socorro solicita una pensión alimenticia de S/ 500.00 mensuales para su hijo Xiana Nairobi Vinces Saucedo, de 2 años de edad, a ser pagada por Eswin Jossimar Vinces Olaya, quien es el padre del menor. María del Socorro argumenta que Eswin ha incumplido con sus obligaciones como padre alimentista y que tiene la capacidad económica para contribuir con los gastos del niño. La demanda se fundamenta en el Código Civil y en el Código de Niños y Adolescentes. Se solicita que la demanda sea declarada fundada. El texto también incluye la designación de un abogado para representar a María del Socorro en el proceso.
Estado: El estado del caso es que la demanda de alimentos presentada por María del Socorro Saucedo Saavedra ha sido rechazada.
Se han creado el archivos en texto plano.

### Tiempo: 1m 8.4s (1 caso analizado)


### PROMPTS:
resumenPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado es una serie de documentos legales de los cuales harás un resumen detallando lo más importante."
estadoPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado es una serie de documentos legales. Determina el estado del caso y asegúrate que tu respuesta sea únicamente el estado del caso."

### Resultados:
OVERVIEW: La demandante, Johana Paola Gomez Lescano, presenta una demanda de filiación judicial de paternidad extramatrimonial en representación de su hijo Dayron Jesús Valladolid Gomez. El demandado es Luis Alberto Valladolid Camacho, con domicilio en el Distrito de Tumán, Provincia de Chiclayo y Departamento de Lambayeque.

La demandante solicita que se declare la filiación extramatrimonial del demandado con su hijo y como consecuencia, se ordene al demandado reconocer al menor y pagar una pensión de alimentos mensual y adelantada de S/. 1,000.00 soles, así como una medida de asignación anticipada de alimentos.

La demandante fundamenta su demanda en el hecho de que el demandado se ha negado a reconocer a su hijo, a pesar de los constantes pedidos de la demandante. La demandante ha inscrito al menor en el Registro Civil con los nombres y apellidos del demandado, pero este no ha realizado el reconocimiento de manera espontánea. La demandante también argumenta que el demandado tiene la capacidad económica para cumplir con la pensión de alimentos solicitada.

La demandante presenta como medios probatorios la partida de nacimiento del menor y el documento nacional de identidad del menor.

En cuanto a la competencia, se señala que el Juez de Paz Letrado es competente según la Ley 28457, que regula el proceso de filiación de paternidad extramatrimonial.

En cuanto al monto del petitorio, se solicita una pensión de alimentos mensual y adelantada de S/. 1,000.00 soles.

Se adjuntan como anexos el DNI de la
Estado: El estado del caso es que la demanda de filiación judicial de paternidad extramatrimonial ha sido admitida a trámite por el Juez de Paz Letrado de Familia de Piura. El demandante, Johana Paola Gomez Lescano, ha presentado la demanda en representación de su menor hijo Dayron Jesús Valladolid Gomez. El demandado, Luis Alberto Valladolid Camacho, ha sido notificado de la demanda y se le ha otorgado un plazo para presentar su descargo. El caso se encuentra en proceso y aún no se ha emitido una resolución definitiva.
Se han creado el archivos en texto plano.
__________________________________________
OVERVIEW: Resumen:

El documento es un seguimiento de un expediente judicial en el Juzgado de Paz Letrado de la Sede de Chipe en el Poder Judicial del Perú. El expediente tiene el número 00201-2022-0-2001-JP-FC-07 y se inició el 3 de febrero de 2022 a las 22:38:38. El motivo de la demanda es por alimentos.

El juez a cargo del caso es Elías Lequernaque Carmen Irina Ocaña Malpartida. El demandante es Juan Manuel Cruz Chinchay y el demandado es Magaly del Milagro Cruz Flores.

El documento menciona que se adjuntan elementos de prueba y que el proceso se encuentra en estado de incidente. No se proporciona información adicional sobre la resolución o el resultado del caso.

La ubicación del expediente es en la Sede de Chipe y la suma del archivo modular es de 5209.31 páginas.
Estado: El estado del caso es que se ha interpuesto una demanda de alimentos por parte de Noe Cruz Chinchay contra Magaly del Milagro Cruz Flores.
Se han creado el archivos en texto plano.

### Tiempo: 1m 22s (2 casos analizados)

### PROMPTS:
resumenPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado son documentos legales de los cuales harás un resumen detallando las personas implicadas, y que es lo que demanda o defiende cada uno."
estadoPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado es una serie de documentos legales. Determina el estado del caso, asegúrate que tu respuesta sea única y exclusivamente el estado del caso. Es decir, tu respuesta debe limitarse a: RECHAZADO, INADMISIBLE, EJECUCION, TRAMITE, PLAZO DE IMPUGNACION, CALIFICACION, ARCHIVO DEFINITIVO, ADMITIDA, ATENDIDO, SENTENCIADO, RESUELTO "

### Resultados:
OVERVIEW: En el presente caso, Johana Paola Gómez Lescano, identificada con DNI Nro. 46626297, demanda la filiación judicial de paternidad extramatrimonial de su hijo Dayron Jesús Valladolid Gómez, de tres meses de edad. El demandado es Luis Alberto Valladolid Camacho, con domicilio en Calle Cruz del Calvario N 1585 del Distrito de Tumán, Provincia de Chiclayo y Departamento de Lambayeque.

La demandante solicita que el demandado reconozca a su hijo y que se le ordene el pago de una pensión de alimentos mensual y adelantada de S/. 1,000.00 soles, así como una medida de asignación anticipada de alimentos.

En cuanto a los fundamentos de hecho, la demandante señala que pese a sus constantes pedidos, el demandado se ha negado a reconocer a su hijo. Por ello, inscribió al menor en el Registro Civil con los nombres y apellidos del demandado. Además, argumenta que su hijo tiene necesidades especiales de alimentación, atención médica y educación, y que ella no cuenta con los recursos económicos suficientes para cubrir todos los gastos.

En cuanto a los fundamentos de derecho, la demandante se ampara en la Ley 28457, Ley que regula el proceso de filiación de paternidad extramatrimonial, y en el artículo 92 del Código de los Niños y Adolescentes. También menciona los artículos 472, 474, 481 y 487 del Código Civil.

Como medios probatorios, la demandante presenta la partida de
Estado: TRAMITE
__________________________________________
OVERVIEW: En el texto proporcionado, se identifican las siguientes personas implicadas:

- Demandante: Carmen Irina Ocaña Malpartida
- Demandado: Juan Manuel Principal Sentenciado/Resuelto
- Demandante Natural: Cruz Chinchay, Noe Cruz Flores, Magaly del Milagro

La demanda presentada es de alimentos, y se adjuntan elementos de prueba. No se especifica cuál es la demanda o defensa de cada una de las partes implicadas.
Estado: TRAMITE
__________________________________________

### Tiempo: 56.7s (2 casos analizados)

## Experimentacion utilizando la funcion IA2():

### PROMPTS:
resumenPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado son documentos legales de los cuales harás un resumen detallando las personas implicadas, y que es lo que demanda o defiende cada uno."
estadoPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado es una serie de documentos legales. Determina el estado del caso, sino puedes determinarlo responde en blanco o espacios, asegúrate que tu respuesta sea única y exclusivamente el estado del caso. Es decir, tu respuesta debe limitarse a: RECHAZADO, INADMISIBLE, EJECUCION, TRAMITE, PLAZO DE IMPUGNACION, CALIFICACION, ARCHIVO DEFINITIVO, ADMITIDA, ATENDIDO, SENTENCIADO, RESUELTO "

### Resultados: 
OVERVIEW: En el texto proporcionado se presenta una demanda de filiación judicial de paternidad extramatrimonial. La demandante principal es Reyes Criollo Kaala Soledad, quien actúa en representación de su hijo. La demanda se dirige contra Luis Alberto Valladolid Camacho. El motivo de la demanda es la filiación y alimentos.

La demandante solicita que el demandado reconozca a su hijo como pretensión principal. Además, solicita que se ordene al demandado pagar una pensión de alimentos mensual y adelantada, así como una medida de asignación anticipada de alimentos.

La demandante fundamenta su demanda en la Ley 28457, Ley que regula el proceso de filiación de paternidad extramatrimonial, así como en el artículo 92 del Código de los Niños y Adolescentes y en diversos artículos del Código Civil.

Como medios probatorios, la demandante presenta la partida de nacimiento del menor y su documento nacional de identidad, entre otros documentos.

En resumen, la demandante busca que se declare la filiación extramatrimonial del demandado con su hijo y que se le ordene pagar una pensión de alimentos mensual y adelantada. También solicita una medida de asignación anticipada de alimentos.
Estado: TRAMITE

### Tiempo: 1m 48.7s (1 caso)

## Experimentacion utilizando la funcion IA3():

### PROMPTS:
resumenPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado son documentos legales de los cuales harás un resumen detallando las personas implicadas, y que es lo que demanda o defiende cada uno."
estadoPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado es una serie de documentos legales. Determina el estado del caso, sino puedes determinarlo responde en blanco o espacios, asegúrate que tu respuesta sea única y exclusivamente el estado del caso. Es decir, tu respuesta debe limitarse a: RECHAZADO, INADMISIBLE, EJECUCION, TRAMITE, PLAZO DE IMPUGNACION, CALIFICACION, ARCHIVO DEFINITIVO, ADMITIDA, ATENDIDO, SENTENCIADO, RESUELTO "

### Resultados: 

OVERVIEW: En el expediente presentado, Johana Paola Gómez Lescano demanda la filiación judicial de paternidad extramatrimonial de su hijo Dayron Jesús Valladolid Gómez, de tres meses de edad. El demandado es Luis Alberto Valladolid Camacho.

La demandante solicita que el demandado reconozca a su hijo y que se le ordene pagar una pensión de alimentos mensual y adelantada de S/. 1,000.00 soles, así como una medida de asignación anticipada de alimentos en la misma cantidad.

La demandante fundamenta su demanda en el hecho de que el demandado se ha negado a reconocer a su hijo a pesar de los constantes pedidos, y ha inscrito al niño en el Registro Civil sin su consentimiento. Además, argumenta que su hijo tiene necesidades especiales de alimentación y cuidado, y que ella no cuenta con los recursos económicos suficientes para cubrir todos los gastos.

La demandante también menciona que el demandado tiene capacidad económica para cumplir con la pensión de alimentos y que las labores domésticas que realiza también constituyen un aporte económico.

En cuanto a la competencia, la demandante señala que el Juez de Paz Letrado es competente según la Ley 28457, que regula el proceso de filiación de paternidad extramatrimonial.

Como medios probatorios, la demandante presenta la partida de nacimiento del menor y su documento nacional de identidad, y solicita que se admita la demanda y se declare fundada en todos sus extremos.

Estado: En blanco.

### Tiempo: 1m 28.0s (1 caso)

# Código de Limpieza de Data

##### El principal problema en la data, es que hay varios archivos PDF que contienen solamente imagenes, o que son escaneos de documentos en físico. Esto genera que el Código de Generación de Dataset genere archivos tipo .txt totalmente en blanco o en los cuales la mayoría de caracteres son espacios, lo que es indeseable.
Por este motivo se ha realizado el codigo de limpieza, que elimina aquellos archivos cuya longitud sea 0, o si al menos el 50% de sus caracteres son espacios.

In [ ]:
rutaTexto= rutaTexto

lista_carpetas = [nombre for nombre in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, nombre))]

for carpeta in lista_carpetas:
    ruta_carpeta=  os.path.join(rutaTexto,carpeta)
    lista_expedientes= [carpeta for carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, carpeta))]

    for expediente in lista_expedientes:
         ruta_expediente = os.path.join(ruta_carpeta,expediente)
         lista_archivos= [nombre for nombre in os.listdir(ruta_expediente)]

         for archivo in lista_archivos:
              ruta_archivo = os.path.join(ruta_expediente,archivo)
              if (ruta_archivo.lower().endswith('.txt')):
                with open(ruta_archivo) as archivo:
                            contenido = archivo.read()
                espacios = sum(1 for c in contenido if c.isspace())
                if len(contenido) == 0:
                    os.remove(ruta_archivo)
                elif (espacios / len(contenido) )>= 0.5 :
                    os.remove(ruta_archivo)





# Código de Métrica Accuracy

In [ ]:
rutaTexto= rutaTexto

lista_carpetas = [nombre for nombre in os.listdir(rutaTexto) if os.path.isdir(os.path.join(rutaTexto, nombre))]
lista_estados= []
lista_estados_deterministicos=[]

for carpeta in lista_carpetas:
    ruta_carpeta=  os.path.join(rutaTexto,carpeta)
    lista_expedientes= [carpeta for carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, carpeta))]

    for expediente in lista_expedientes:
         ruta_expediente = os.path.join(ruta_carpeta,expediente)
         lista_IA= [nombre for nombre in os.listdir(ruta_expediente) if os.path.isdir(os.path.join(ruta_expediente, nombre))]

         for nombre in lista_IA:
            ruta_IA=  os.path.join(ruta_expediente,nombre)
            lista_final = [nombre for nombre in os.listdir(ruta_IA) ]

            for nombre in lista_final:
                 ruta_final = os.path.join(ruta_IA,nombre)


                 if ruta_final.lower().endswith('estado.txt'):
                    with open(ruta_final, "r") as archivo:
                            estado= archivo.read()
                    lista_estados.append(estado)
                 elif ruta_final.lower().endswith('estado_deterministico.txt'):
                    with open(ruta_final, "r") as archivo:
                            estado_deterministico= archivo.read()
                    lista_estados_deterministicos.append(estado_deterministico)
print(lista_estados)
print(lista_estados_deterministicos)

In [ ]:
#Esta funcion es para corregir los casos deterministicos en caso haya un error de tipeo, se utiliza una lista con todos los estados posibles del caso
def corregir_estado(estado, estados_validos):
    matches = get_close_matches(estado, estados_validos)
    if matches:
        return matches[0]
    else:
        return estado

estados_validos = [
    "RECHAZADO",
    "INADMISIBLE",
    "EJECUCION",
    "TRAMITE",
    "PLAZO DE IMPUGNACION",
    "CALIFICACION",
    "ARCHIVO DEFINITIVO",
    "ADMITIDA",
    "ATENDIDO",
    "SENTENCIADO",
    "RESUELTO"
]
estados_deterministicos_corregidos = [corregir_estado(estado, estados_validos) for estado in lista_estados_deterministicos]
print(estados_deterministicos_corregidos)
#Ojo: resuelto atendido y sentenciado es el mismo estado, pero con diferente designacion.

In [ ]:
#Esta funcion es para extraer solo el estado de toda la respuesta predicha por la IA, para versiones posteriores se considera utilizar esta funcion para generar el archivo de texto
#En este caso no se utiliza, porque falta informacion acerca de TODOS los estados posibles existentes.
def extraer_estado(cadena, estados_validos):
    for estado in estados_validos:
        if estado.lower()[:-1] in cadena.lower():
            return estado
    return cadena

# Generar la lista de estados predichos2
estados_predichos = [extraer_estado(estado, estados_validos) for estado in lista_estados]
print(estados_predichos)

In [ ]:
#Ahora, ya con los estados extraidos y corregidos, cambiamos normalizamos los elementos resueltos, atendidos y sentenciados, por uno solo:
i=0
for elemento in estados_deterministicos_corregidos:
  if elemento == 'ATENDIDO' or elemento == 'RESUELTO':
    estados_deterministicos_corregidos[i]= 'SENTENCIADO'
  i+=1

i=0
for elemento in estados_predichos:
  if elemento == 'ATENDIDO' or elemento == 'RESUELTO':
    estados_predichos[i]= 'SENTENCIADO'
  i+=1

In [ ]:
#Calculamos la métrica de precisión.
total_predicciones = len(lista_estados)
predicciones_correctas = sum(1 for corr, pred in zip(estados_deterministicos_corregidos,estados_predichos) if corr == pred)
precision = predicciones_correctas / total_predicciones

# Imprimir la precisión
print(f"La precisión del modelo es: {precision:.2%}")

In [ ]:
#En esta parte, calculamos datos más específicos del modelo que podrían ayudarnos a saber cuales son los puntos débiles y fuertes.
estados_validos2= [
    "RECHAZADO",
    "INADMISIBLE",
    "EJECUCION",
    "TRAMITE",
    "PLAZO DE IMPUGNACION",
    "CALIFICACION",
    "ARCHIVO DEFINITIVO",
    "ADMITIDO",
    "SENTENCIADO"
]

# Realizar el análisis para cada estado
for estado in estados_validos2:
    estados_correctos_count = estados_deterministicos_corregidos.count(estado)
    estados_predichos_count = estados_predichos.count(estado)
    estados_correctamente_predichos = sum(1 for predicho in estados_predichos if predicho == estado and predicho in estados_deterministicos_corregidos)
    no_estados_correctamente_predichos = sum(1 for predicho in estados_predichos if predicho == estado and predicho not in estados_deterministicos_corregidos)
    print(f"Para {estado}:")
    print(f"La red neuronal predijo correctamente {estados_correctamente_predichos} estados {estado} de un total de {estados_correctos_count}.")
    print(f"Además, detectó como {estado} {no_estados_correctamente_predichos} casos que no lo eran.")

# Conclusiones

- En cuanto a velocidad de procesamiento, la mejor forma de introducir la data es usando la función IA1, ya que esta solo hace uso del modelo  una sola vez por caso, mientras que el resto de funciones hacen uso del modelo una vez por documento.

- Las funciones IA2 e IA3 empiezan con muy poca información (ya que empiezan leyendo un solo archivo de todo el caso), esto genera que al principio, sus respuestas sean que no se pudo determinar el estado o hacer un resumen adecuado, esto genera que para las siguientes iteraciones, el modelo tenga acceso a información contradictoria, lo que genera confusión y alucinaciones en algunos casos.

- Los mejores prompts encontrados hasta ahora fueron: resumenPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado son documentos legales de los cuales harás un resumen detallando las personas implicadas, y que es lo que demanda o defiende cada uno."
estadoPrompt= "Eres un experto en Asesoría legal, el contexto proporcionado es una serie de documentos legales. Determina el estado del caso, asegúrate que tu respuesta sea única y exclusivamente el estado del caso. Es decir, tu respuesta debe limitarse a: RECHAZADO, INADMISIBLE, EJECUCION, TRAMITE, PLAZO DE IMPUGNACION, CALIFICACION, ARCHIVO DEFINITIVO, ADMITIDA, ATENDIDO, SENTENCIADO, RESUELTO "

- El uso del API del modelo GPT 3.5 de openAI fue un limitante, ya que este tiene un limite determinado de tokens y un numero determinado de usos por minuto, lo mejor hubiera sido poder alojar y ejecutar nuestro propio modelo LLM, lo cual no se pudo hacer en esta ocasión por falta de recursos computacionales.

- Como se mencionó anteriormente, el limite de tokens entorpeció la tarea asignada, sin embargo existen técnicas como el split y overlapping que permiten disminuir las dificultades del límite de tokens. Estas técnicas están siendo estudiadas y se intentará incluirlas en los próximos entregables. 